# Football Betting with Multinomial Logit

* **Data Source:** [https://www.kaggle.com/hugomathien/soccer](https://www.kaggle.com/hugomathien/soccer)
* **Author:** Anders Munk-Nielsen

In [1]:
import pandas as pd 
import numpy as np 
from scipy.optimize import minimize 
import statsmodels.formula.api as smf 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()

In [2]:
# Read
dat = pd.read_csv('football_probs.csv')

# Data types 
dat.date = pd.to_datetime(dat.date)
cols_to_cat = ['league', 'season', 'team', 'country']
for c in cols_to_cat: 
    dat[c] = dat[c].astype('category')

# Outcome

The outcome, $y$, can take three values, $y \in \{ 0, 1, 2\}$, corresponding to Lose, Draw, and Win, respectively. 

In [3]:
dat['y'] = 0 # loss
dat.loc[dat.goal_diff == 0.0 , 'y'] = 1 # draw 
dat.loc[dat.goal_diff >  0.0 , 'y'] = 2 # win 

assert (dat.loc[dat.goal_diff < 0.0 , 'y'] == 0).all()

# Conditional Probabilities

We assume that the probability that $y$ takes each of the values is given by 
$$ \Pr (y_i = j | \mathbf{X}_i, \theta) = \frac{\exp(\mathbf{x}_{ij} \theta_j)}{\sum_{k=0,1,2} \exp(\mathbf{x}_{ik} \theta_k)}. $$

In other words, we can think of $v_{ij} \equiv \mathbf{x}_{ij}\theta_j$ as the scalar index that determines the relative probabilities: the higher is $v_{i1}$ (for fixed $v_{i0},v_{i2}$), the more likely it is that the outcome is $y_i = 1$. 

Compared to other models we have seen, we now have $\mathbf{x}_{ij}$ varying both over $i$ and $j$. Previously, we have otherwise looked at the *Multinomial Logit* model ($\mathbf{x}$ only varies with $i$, e.g. years of schooling, or alcohol abuse), or the *Conditional Logit* model ($\mathbf{x}$ only varies with $j$, e.g. car attributes). In this dataset, we will allow for covariates that vary over $i$ but which also do not necessarily enter in the "utility" for all outcomes. E.g. we may assume that the Bet365 Probability that the Away team wins, `B365_PrA`, should only affect the probability of Away winning (even though we could allow it to enter in the other utilities. Other regressors, however, like the `home` dummy (for whether the match is played at the home stadium) affects all outcomes, but with different coefficients. 

## Normalization, short version

We have to normalize $\theta_j = \mathbf{0}_{K\times 1}$ for one $j$. We will choose to normalize $j=0$. 
* It does not matter which $j$ we choose as the normalizing alternative, except of course for the interpretation of the coefficients. 

## Estimation  

This implies that our probabilities are 
$$ \Pr(y_i = j| \mathbf{x}_{i1}, \mathbf{x}_{i2}; \boldsymbol{\theta}_1, \boldsymbol{\theta}_2) = 
\begin{cases}
\frac{1}{1+\exp(\mathbf{x}_{i1}\boldsymbol{\theta}_{1})+\exp(\mathbf{x}_{i2}\boldsymbol{\theta}_{2})} & \text{if }j=0\\
\frac{\exp(\mathbf{x}_{ij}\boldsymbol{\theta}_{j})}{1+\exp(\mathbf{x}_{i1}\boldsymbol{\theta}_{1})+\exp(\mathbf{x}_{i2}\boldsymbol{\theta}_{2})} & \text{otherwise.}
\end{cases}
$$

Our full vector of parameters thus consists of $(\boldsymbol{\theta}_1, \boldsymbol{\theta}_2)$, which is $2K$ long. (There are $J-1 = 2$ sets of coefficients, because we have normalized one of our $J=3$ alternatives to have zero coefficients.) 

## Normalization, Longer version

Since probabilities must sum to 100%, we cannot allow $v_{ij}$ to be changing freely. For instance, if we add the same scalar, $\lambda \in \mathbb{R}$, to all indices, then we get unchanged probabilities: 
$$ \frac{\exp(\lambda + v_{ij})}{\sum_{k=0,1,2} \exp(\lambda + v_{i\ell})} = \underset{=1}{\underbrace{\frac{\exp(\lambda)}{\exp(\lambda)}}} \frac{\exp(\mathbf{x}_{ij} \theta_j)}{\sum_{k=0,1,2} \exp(\mathbf{x}_{ik} \theta_k)}. $$ 
This means that we cannot distinguish the implied behavior by the indices $(v_{i0},v_{i1},v_{i2})$, and that implied by $(v_{i0}+\lambda,v_{i1}+\lambda,v_{i2}+\lambda)$: The choice probabilities are analytically identical. This means that we cannot include a common intercept in all $v_{ij}$, which in practice implies that we can have at most $J-1 = 2$ intercepts. Similar arguments will explain why we cannot have all regressors entering in all $v_{ij}$, which amounts to saying that we have to set $\theta_j = \mathbf{0}_K$ for one alternative $j$. 

## Setting up regressors

In [4]:
# add an explanatory variable: home dummy, with separate effect on Pr(D) and Pr(W)
dat['home_W'] = dat.home.copy() * 1.0 
dat['home_D'] = dat.home.copy() * 1.0 

dat['const_W'] = 1.0 
dat['const_D'] = 1.0 

In [5]:
SMALLSCALE = True

if SMALLSCALE: 
    # only Bet365, a home-dummy and a constant term 
    cols = [f'{x}{o}' for x in ['B365_Pr', 'home_', 'const_'] for o in ['D','W']]
    
else: 
    # many variables, but not those that have too many missings
    firms_drop = ['BS', 'GB', 'PS', 'SJ'] # these are missing in too many years 
    cols = [c for c in dat.columns if (c[-4:-1] == '_Pr') # probabilities of win lose draw 
                                    & (c[-1] != 'L') # lose will be the normalized outcome 
                                    & (c[:2] not in firms_drop) # do not include from these firms 
           ] 
    cols += ['home_D', 'home_W', 'const_D', 'const_W']

print(f'Chosen columns: {cols}')

Chosen columns: ['B365_PrD', 'B365_PrW', 'home_D', 'home_W', 'const_D', 'const_W']


Dropping observations (rows) with missing values for one or more variables. 

In [6]:
I = dat[cols].notnull().all(1)
print(f'Keeping only non-missings: {I.mean():5.2%} obs. (N = {I.sum():,d})')
dat = dat.loc[I, :].copy()

Keeping only non-missings: 86.96% obs. (N = 45,184)


## Pandas to numpy 

We want to write our objective function in `numpy`, so we have to extract our variables. 

In [7]:
cols

['B365_PrD', 'B365_PrW', 'home_D', 'home_W', 'const_D', 'const_W']

In [8]:
J = 3 # 3 discrete choices: Lose, Draw, Win 
K = round(len(cols) / (J-1)) # J-1 since we only have covariates for the two 
N = dat.shape[0]

print(f'Chosen columns: {cols}')

y = dat['y'].values.reshape((N,))
X = dat[cols].values.reshape((N,K,J-1))

assert not np.any(np.isnan(X)), 'NaNs in X'
assert not np.any(np.isnan(y)), 'NaNs in y'

Chosen columns: ['B365_PrD', 'B365_PrW', 'home_D', 'home_W', 'const_D', 'const_W']


## Visualizing `X` and working with 3D arrays 

In [9]:
ii = [0,1,-10] # let's look at these rows
x = X[ii]

# and print them nicely from pandas using "iloc"
dat.iloc[ii][cols + ['team', 'enemy_team']]

,B365_PrD,B365_PrW,home_D,home_W,const_D,const_W,team,enemy_team
0,0.274324,0.539135,1.0,1.0,1.0,1.0,KRC Genk,Beerschot AC
1,0.283293,0.464891,1.0,1.0,1.0,1.0,SV Zulte-Waregem,Sporting Lokeren
50526,0.094919,0.049957,0.0,0.0,1.0,1.0,SD Eibar,FC Barcelona


In [10]:
theta = 0.1 * np.ones((K,J-1))

$\theta$: one parameter for each $(k,j)$, but $\theta_{jk}=0 $ for the normalized alternative, $j=0$. 

In [11]:
theta

array([[0.1, 0.1],
       [0.1, 0.1],
       [0.1, 0.1]])

In [12]:
# labels? 
pd.DataFrame(np.reshape(cols, (K,J-1)), columns=['Draw', 'Win'])

,Draw,Win
0,B365_PrD,B365_PrW
1,home_D,home_W
2,const_D,const_W


$x_{ijk}$: for a given $i$, $K\times (J-1)$. 

In [13]:
i = 2
x[i] 

array([[0.09491876, 0.04995724],
       [0.        , 0.        ],
       [1.        , 1.        ]])

$x_{ijk} \beta_{jk}$

In [14]:
xb = x[i] * theta
xb

array([[0.00949188, 0.00499572],
       [0.        , 0.        ],
       [0.1       , 0.1       ]])

$v_{ij} \equiv \sum_k x_{ijk} \theta_{jk}$  (one for each $j=1,2$)

In [15]:
xbsum = xb.sum(axis=0)
xbsum

array([0.10949188, 0.10499572])

$v_{i0} := 0$ (normalization)

In [16]:
u = np.hstack([0., xbsum])
u

array([0.        , 0.10949188, 0.10499572])

In [17]:
denom = np.sum(np.exp(u))
ccp = np.exp(u) / denom 
ccp

array([0.30994135, 0.34580498, 0.34425367])

### With three individuals

In [18]:
theta

array([[0.1, 0.1],
       [0.1, 0.1],
       [0.1, 0.1]])

In [19]:
xb = x * theta
xb

array([[[0.02743245, 0.05391348],
        [0.1       , 0.1       ],
        [0.1       , 0.1       ]],

       [[0.0283293 , 0.0464891 ],
        [0.1       , 0.1       ],
        [0.1       , 0.1       ]],

       [[0.00949188, 0.00499572],
        [0.        , 0.        ],
        [0.1       , 0.1       ]]])

### Scale up to the full dataset

In [20]:
theta.shape

(3, 2)

In [21]:
X.shape

(45184, 3, 2)

In [22]:
(X * theta).sum(axis=1).shape

(45184, 2)

$X$ is $N \times K \times J-1$, so we sum over the middle-index $k$, `axis=1`. 

In [23]:
xb = (X * theta).sum(axis=1) # sum over J-dimension
xb

array([[0.22743245, 0.25391348],
       [0.2283293 , 0.2464891 ],
       [0.22788191, 0.23865979],
       ...,
       [0.12389025, 0.11365157],
       [0.12929936, 0.12929936],
       [0.12771654, 0.12944882]])

In [24]:
u = np.hstack([np.zeros((N,1)), xb])
u 

array([[0.        , 0.22743245, 0.25391348],
       [0.        , 0.2283293 , 0.2464891 ],
       [0.        , 0.22788191, 0.23865979],
       ...,
       [0.        , 0.12389025, 0.11365157],
       [0.        , 0.12929936, 0.12929936],
       [0.        , 0.12771654, 0.12944882]])

To max rescale, compute maximum over rows, i.e. `axis=1`

Choice probabilities 

In [25]:
denom = np.sum(np.exp(u), axis=1).reshape(N,1)

In [26]:
(np.exp(u) / denom)

array([[0.28213258, 0.35418152, 0.3636859 ],
       [0.28280349, 0.35534231, 0.3618542 ],
       [0.28364903, 0.35624532, 0.36010565],
       ...,
       [0.30747912, 0.34803305, 0.34448783],
       [0.30524457, 0.34737771, 0.34737771],
       [0.3053965 , 0.34700094, 0.34760256]])

_____
# Criterion function

Our estimator is defined as 
$$ \hat{\theta} = \arg\min_\theta \frac{1}{N} \sum_{i=1}^N q_i(\theta),$$

where the criterion function is 
$$ q_i(\theta) \equiv - \log \Pr(y_i \vert x_i, \theta),$$
and choice probabilities are computed as 
$$ \Pr(y_i \vert x_i, \theta) = \frac{\exp(v_{iy_i})}{\sum_{j=1}^J \exp(v_{ij})}, $$
and utility indices are given by 
$$ v_{ij} \equiv \mathbf{x}_{ij} \boldsymbol{\beta}_j = \sum_{k=1}^K \beta_{jk} x_{ijk}.$$
We make the *normalization* that $\boldsymbol{\beta}_1 := \mathbf{0}_{K \times 1}$. Thus, we need $J-1$ regressors (with $K$ variables each) and we are estimating $(J-1)K$ parameters. 

In reality, the choice probabilities are computed using a "max-rescaling," 
$$ \Pr(y_i \vert x_i, \theta) = \frac{\exp(v_{iy_i} - K_i)}{\sum_{j=1}^J \exp(v_{ij} - K_i)},\quad 
K_i \equiv \max_{j \in \{1,...,J\}} v_{ij}.$$

In [27]:
def util(x, thet) -> np.ndarray: 
    '''util: Utility function. Explicitly implements the normalization 
             for the first alternative (i.e. u[:, 0] = 0).
    Args. 
        x: (N,J-1,K)-matrix of explanatory variables
        theta: (J-1,K)-matrix of coefficients 
        
    Returns
        u: (N,J)-matrix of utilities (max-rescaled)
    '''
    N,K,J_1 = x.shape
    J = J_1 + 1 
    assert thet.size == K*(J-1)
    theta = thet.reshape((K, J-1)) # minimizer may flatten this 
        
    xb = (x * theta).sum(axis=1) # (N,K,J-1) * (K,J-1), sum over k-axis -> (N,J-1)
    oo = np.zeros((N,1)) # normalized alternative
    
    u = np.hstack([oo, xb]) # full N*J matrix of utilities 
    
    u -= u.max(axis=1).reshape((N,1)) # max rescale
    
    return u 

In [28]:
def ccp(x, theta): 
    N,K,J_1 = x.shape
    u = util(x, theta) # (N,J)
    denom = np.sum(np.exp(u), axis=1).reshape((N,1)) # (N,1)
    ccp = np.exp(u) / denom # (N,J) matrix
    return ccp 

In [29]:
def loglike(y, x, theta): 
    N,K,J_1 = x.shape
    J = J_1 + 1 
    assert np.isin(y, np.arange(J)).all()
    
    u = util(x, theta)
    denom = np.sum(np.exp(u), axis=1).reshape((N,1))
    
    u_i = np.take_along_axis(u, y.reshape((N,1)), axis=1)
    
    ll_i = u_i - np.log(denom)
    return ll_i

In [30]:
def Q(theta): 
    ll_i = loglike(y, X, theta) # reads y,x from global memory 
    return -np.mean(ll_i)

In [31]:
theta0 = np.zeros((K,J-1))

In [32]:
res = minimize(Q, theta0)

print(f'Convergence success? {res.success} (after {res.nit} iterations).')

pd.DataFrame(res.x, index=cols, columns=['beta'])

Convergence success? True (after 42 iterations).


,beta
B365_PrD,5.311748
B365_PrW,4.919429
home_D,0.416425
home_W,0.205177
const_D,-1.971006
const_W,-1.982278


In [33]:
theta = res.x.reshape((K,J-1))

In [34]:
pd.DataFrame(theta, columns=['Draw', 'Win'])

,Draw,Win
0,5.311748,4.919429
1,0.416425,0.205177
2,-1.971006,-1.982278


## Inspect fit for individual rows 

This will print out the real outcomes, `y`, as well as the predicted probabilities of each outcome, where `0` denotes `L`, `1` denotes `D`, and `2` denotes `W` (Loss, Draw, Win, respectively). 

In [35]:
probs = ccp(X, res.x)
prob_i = np.take_along_axis(probs, y.reshape(N,1), axis=1)

# y is the actual outcome of the game, 
# the other three columns are our predicted probabilities 
tab = pd.DataFrame(np.hstack([y.reshape(-1,1), probs, 
                              dat[['team', 'home', 'B365_PrL', 'B365_PrD', 'B365_PrW']].values.reshape(-1,5) ]
                            ), 
                   columns=  ['y', 'Pr(0)', 'Pr(1)', 'Pr(2)', 'team', 'home', 'B365 P(0)', 'B365 P(1)', 'B365 P(2)']) 
tab.sample(10, random_state=1337).round(3)

,y,Pr(0),Pr(1),Pr(2),team,home,B365 P(0),B365 P(1),B365 P(2)
41465,0,0.47747,0.301025,0.221505,Heart of Midlothian,False,0.46896,0.284218,0.246821
39174,2,0.155398,0.059063,0.785539,FC Porto,False,0.078726,0.188942,0.732332
24287,1,0.329398,0.189844,0.480758,RSC Anderlecht,False,0.252871,0.267321,0.479807
39894,0,0.689306,0.18623,0.124464,Académica de Coimbra,False,0.820305,0.124686,0.055009
40457,1,0.27966,0.16227,0.55807,Rangers,False,0.188014,0.268592,0.543394
38325,1,0.47747,0.301025,0.221505,Leixões SC,False,0.46896,0.284218,0.246821
13250,2,0.388303,0.355998,0.255699,N.E.C.,True,0.447368,0.276316,0.276316
26646,1,0.561428,0.267101,0.171471,Crystal Palace,False,0.606936,0.231214,0.16185
28249,0,0.44389,0.326832,0.229278,AJ Auxerre,False,0.41791,0.313433,0.268657
30566,0,0.612603,0.245854,0.141542,Eintracht Frankfurt,False,0.695688,0.199186,0.105126


In [36]:
pr_i = np.take_along_axis(probs, y.reshape(-1,1), 1).flatten()

probs_B365 = dat[['B365_PrL', 'B365_PrD', 'B365_PrW']].values
pr_i_B365 = np.take_along_axis(probs_B365, y.reshape(-1,1), 1).flatten()

Evaluate the probability of the observed outcome from our model vs. B365. 

In [37]:
tab = pd.DataFrame({'Us':pr_i, 'B365':pr_i_B365})
tab

,Us,B365
0,0.210656,0.274324
1,0.263075,0.283293
2,0.326591,0.334583
3,0.684157,0.634518
4,0.441543,0.555115
...,...,...
45179,0.553319,0.604340
45180,0.451761,0.424796
45181,0.566503,0.624582
45182,0.294522,0.292994


In [38]:
tab.mean()

Us      0.410486
B365    0.416424
dtype: float64

In [39]:
avg_prob_point_diff = (tab['Us'] - tab['B365']).mean()
avg_times_closer = (tab['Us'] > tab['B365']).mean()

print(f'On avg. our model as a diff. of {avg_prob_point_diff:5.2%}-points')
print(f'Our model has a higher predicted probability of the observed outcome for {avg_times_closer:5.2%} of matches')
if avg_prob_point_diff<0.0: 
    print(f'*sigh* the model is not doing better than B365...')
else: 
    print(f'Wahoo, we''ve beat the market!')

On avg. our model as a diff. of -0.59%-points
Our model has a higher predicted probability of the observed outcome for 49.00% of matches
*sigh* the model is not doing better than B365...


How is it possible that our model performs *worse*? Well, it's maximizing the avg. *log* probability: 

In [40]:
tab.mean().to_frame('Avg. Probability')

,Avg. Probability
Us,0.410486
B365,0.416424


In [41]:
np.log(tab).mean().to_frame('Avg. log Pr')

,Avg. log Pr
Us,-0.990071
B365,-0.971868
